<a href="https://colab.research.google.com/github/mhirschberg/py_demo/blob/main/FreeAImode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required dependencies
# - requests: For making HTTP calls to Bright Data APIs
!pip install -q requests
# Import required libraries
import requests
import json
import time
from rich.console import Console
from rich.markdown import Markdown

In [2]:
def poll_and_download_snapshot(snapshot_id, token):
    monitor_url = f"https://api.brightdata.com/datasets/v3/progress/{snapshot_id}"
    download_url = f"https://api.brightdata.com/datasets/v3/snapshot/{snapshot_id}"

    headers = {"Authorization": f"Bearer {token}"}

    while True:
        response = requests.get(monitor_url, headers=headers)
        status = response.json().get("status")

        if status == "ready":
            download_response = requests.get(download_url, headers=headers)
            download_data = download_response.json()

            # Check if download is actually ready or still building
            if download_data.get("status") == "building":
                time.sleep(10)
                continue

            print(download_response.text)
            return download_data

        time.sleep(10)

def format_response(data):
    if isinstance(data, str):
        import json
        data = json.loads(data)

    console = Console()

    console.print("[bold cyan]PROMPT:[/bold cyan]")
    console.print(data.get('prompt', ''))
    print()

    console.print("[bold cyan]ANSWER:[/bold cyan]")

    answer = data.get('answer_text_markdown', data.get('answer_text', ''))
    md = Markdown(answer)
    console.print(md)
    print()
    console.print("[bold cyan]CITATIONS:[/bold cyan]")

    for i, citation in enumerate(data.get('citations', []), 1):
        console.print(f"[bold]{i}.[/bold] {citation.get('title', 'N/A')} - [dim]{citation.get('domain', 'N/A')}[/dim]")
        console.print(f"   [link={citation.get('url', 'N/A')}]{citation.get('url', 'N/A')}[/link]")
        console.print()

In [3]:
#@title 👉 Enter Your Prompt Here
#@markdown ---
#@markdown Enter your prompt below, then run the rest of the notebook.
#@markdown ---

MY_PROMPT = "What day is it today?" #@param {type:"string"}
MY_COUNTRY = "US" #@param {type:"string"}

# ============================================
# API credentials (from Colab secrets)
# ============================================
from google.colab import userdata
import os

# API KEY
os.environ['BRIGHTDATA_API_TOKEN'] = userdata.get('BRIGHTDATA_API_TOKEN')
BRIGHTDATA_API_TOKEN = os.environ.get("BRIGHTDATA_API_TOKEN")

print("Configuration loaded:")
print(f"  Prompt: {MY_PROMPT}")
print(f"  Country: {MY_COUNTRY}")
print(f"  Bright Data API: {'configured' if BRIGHTDATA_API_TOKEN else 'X missing'}")

Configuration loaded:
  Prompt: What day is it today?
  Country: US
  Bright Data API: configured


In [ ]:
headers = {
    "Authorization": f"Bearer {BRIGHTDATA_API_TOKEN}",
    "Content-Type": "application/json",
}

data = json.dumps({
    "input": [{"url": "https://google.com/aimode", "prompt": MY_PROMPT, "country": MY_COUNTRY}],
})

response = requests.post(
    "https://api.brightdata.com/datasets/v3/scrape?dataset_id=gd_mcswdt6z2elth3zqr2&custom_output_fields=prompt%2Canswer_text%2Ccitations%2Canswer_text_markdown%2Ctimestamp&notify=false&include_errors=true",
    headers=headers,
    data=data
)

bd_answer = (response.json())

if 'snapshot_id' in bd_answer:
    snapshot_id = bd_answer['snapshot_id']
    bd_answer = poll_and_download_snapshot(snapshot_id, BRIGHTDATA_API_TOKEN)

format_response(bd_answer)